# Практическая работа №1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Загрузим данные для работы. 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Elections.csv to Elections.csv


In [ ]:
df = pd.read_csv("Elections.csv",delimiter=';')
df.head()

,Region Index,Subjects of Russia,Unemployment rate. %,Income per capita. RUB,Average salary. RUB per month,Spendings per capita. RUB,State housing. %,Higher Education. %,Number of doctors per 10000 citizens,GRP per capita. RUB,Investment in total capital,Consumer Price Index (CPI),Average income / min. living wage,Average costs / min. living wage,Votes for United Russia. %
0,1,Adygea,43505.0,12236,12787.4,8361,0.6,4.0,37.9,93548.3,25834,110.1,42036,17168.0,61.002
1,2,Altai,43685.0,10926,12050.7,8139,43680.0,43694.0,48.7,107282.4,23048,108.2,33604,15707.0,37.174
2,3,Amur Oblast,43714.0,14064,21207.5,9406,2.0,43590.0,62.9,175927.8,95619,109.4,17199,23743.0,43.543
3,4,Arkhangelsk Oblast,43503.0,19310,22192.1,11688,43527.0,9.0,56.5,256728.4,63912,109.4,14305,43587.0,31.900
4,5,Astrakhan Oblast,43532.0,14641,16582.1,10651,43586.0,43472.0,68.2,131400.1,56338,109.8,20852,31778.0,60.170


Будем решать задачу регрессии: необходимо предсказать процент голосов за Единую Россию в каждом регионе по различным характеристикам уровня жизни в регионе.

## Шаг 1. 
Создайте матрицу X объект-признак и целевой вектор y ("Votes for United Russia. %")

In [ ]:
X = df.copy()
y = df['Votes for United Russia. %']
del X['Votes for United Russia. %']

## Шаг 2.
Оставьте в матрице X только столбцы 'Unemployment rate. %', 'Spendings per capita. RUB', 'Number of doctors per 10000 citizens', 'Average salary. RUB per month'

In [ ]:
cols = ['Region Index', 'Subjects of Russia', 'Income per capita. RUB', 'State housing. %', 'Higher Education. %','GRP per capita. RUB', 'Investment in total capital', 'Consumer Price Index (CPI)', 'Average income / min. living wage', 'Average costs / min. living wage']
X.drop(cols, inplace=True, axis =1)

In [ ]:
X.head()

,Region Index,Subjects of Russia,Unemployment rate. %,Income per capita. RUB,Average salary. RUB per month,Spendings per capita. RUB,State housing. %,Higher Education. %,Number of doctors per 10000 citizens,GRP per capita. RUB,Investment in total capital,Consumer Price Index (CPI),Average income / min. living wage,Average costs / min. living wage
0,1,Adygea,43505.0,12236,12787.4,8361,0.6,4.0,37.9,93548.3,25834,110.1,42036,17168.0
1,2,Altai,43685.0,10926,12050.7,8139,43680.0,43694.0,48.7,107282.4,23048,108.2,33604,15707.0
2,3,Amur Oblast,43714.0,14064,21207.5,9406,2.0,43590.0,62.9,175927.8,95619,109.4,17199,23743.0
3,4,Arkhangelsk Oblast,43503.0,19310,22192.1,11688,43527.0,9.0,56.5,256728.4,63912,109.4,14305,43587.0
4,5,Astrakhan Oblast,43532.0,14641,16582.1,10651,43586.0,43472.0,68.2,131400.1,56338,109.8,20852,31778.0


## Шаг 3.
Разбейте данные на train и test (доля тестовых данных - 30%).

In [ ]:
ind_train, ind_test = train_test_split(range(len(X)), test_size=0.3, random_state=0)
X_train = X.iloc[ind_train]
y_train = y[ind_train]
X_test = X.iloc[ind_test]
y_test = y[ind_test]

## Шаг 4.
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [ ]:
scaler = StandardScaler().fit(X_train)
X_scaled=scaler.transform(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_test)

## Шаг 5.
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [ ]:
print('MSE: train =', mean_squared_error(y_train, y_pred_train), ', test =', mean_squared_error(y_test, y_pred_test))
print('R2: train =', r2_score(y_train, y_pred_train), ', test =', r2_score(y_test, y_pred_test))

MSE: train = 230.13873586087146 , test = 247.48618961500324
R2: train = 0.17739672347911017 , test = 0.13827291604639314


## Шаг 6.
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [ ]:
print('R2 =', cross_val_score(LinearRegression(), X, y, cv=5, scoring='r2').mean())

R2 = -0.27874090603166357


## Шаг 7.
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [ ]:
from sklearn.linear_model import Lasso

r2_best=0
for a in np.arange(0.1, 1.1, 0.1):
    regressor2 = Lasso(alpha=a)
    regressor2.fit(X_train, y_train)
    y_pred2 = regressor2.predict(X_train)
    r2_now = r2_score(y_train, y_pred2)
    if r2_best<r2_now:
      r2_best = r2_now
      best_estimation = regressor2.coef_

## Шаг 8.
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [ ]:
print(best_estimation)
print(r2_best)

[-4.63861284  3.81315838 -3.94917155  2.48943014]
0.17709038277964817


## Шаг 9.

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [ ]:
y_pred2 = regressor2.predict(X_test)
print('R2 =', r2_score(y_test, y_pred2))


R2 = 0.1563671804782788


## Шаг 10.

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

pf = PolynomialFeatures(2)
X_polynomial = pf.fit_transform(X_scaled)
X_train_poly = pf.fit_transform(X_train)
X_test_poly = pf.fit_transform(X_test)

print('R2 =', cross_val_score(LinearRegression(), X_polynomial, y, cv=5, scoring='r2').mean())


R2 = -178880.74682007037


## Шаг 11.
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train_poly, y_train)

y_pred_train = regressor.predict(X_train_poly)
y_pred_test = regressor.predict(X_test_poly)

print('MSE: train =', mean_squared_error(y_train, y_pred_train), ', test =', mean_squared_error(y_test, y_pred_test))
print('R2: train =', r2_score(y_train, y_pred_train), ', test =', r2_score(y_test, y_pred_test))

MSE: train = 129.13983280293365 , test = 340.71353377660756
R2: train = 0.538405174619217 , test = -0.18633722706540023


## Сделайте выводы. Для этого ответьте на вопросы:

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

In [ ]:
1) Исходная модель получилась недостаточно хорошая, поскольку R2 небольшой.
В модели наблюдается незначительное переобучение, о чем можно судить по сократившемуся R2 на тестовых данных по сравнению с тренировачными данными.
2) L1-регуляризация не помогла улучшить исходную модель - R2 получился отрицательным.
3) Добавление полиномов второй степени так же не принесло серьёзных улучшений - хотя для тренировачных данных R2 возрос более чем на половину, для тестовых данных он отрицателен. 

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [ ]:
#your code here